In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [ ]:
# Load the DataFrames from joblib
df = joblib.load('df.joblib')

In [ ]:
df_spread_time_deterministic = joblib.load('df_spread_time_deterministic.joblib')
df_spread_time_constant = joblib.load('df_spread_time_log_constant.joblib')
df_spread_time_first = joblib.load('df_spread_time_log_first.joblib')

In [ ]:
df = df[df['eta'] < 23]

In [ ]:
def compute_proportion_q(df_cur):
    df_cur = df_cur[df_cur['Utilization'] != 'N']
    pivot_table = df_cur.pivot_table(index='Tie range', columns='Utilization', aggfunc='size', fill_value=0)
    pivot_table['Total'] = pivot_table.sum(axis=1)
    
    # Ensure 'Q' column exists as a Series
    if 'Q' not in pivot_table.columns:
        pivot_table['Q'] = 0

    pivot_table['Proportion_Q'] = pivot_table['Q'] / pivot_table['Total']
    pivot_table.reset_index(inplace=True)
    return pivot_table[['Tie range', 'Proportion_Q']]



# Compute proportions for each model
df_d = compute_proportion_q(df[df['Type'] == 'Deterministic'].copy())
df_n = compute_proportion_q(df[df['Type'] == 'Constant'].copy())
df_i = compute_proportion_q(df[df['Type'] == 'First'].copy())

In [ ]:
COLOR_DETERMINISTIC = '#1f77b4'
COLOR_CONSTANT = '#ffa600'
COLOR_FIRST = '#ff7c43'

In [ ]:
# Set up the subplots
fig, axes = plt.subplots(1, 1, figsize=(7, 4), sharex=True)

# Plot for non-ICM
sns.histplot(data=df[df['Type'] == 'Constant'], 
             x='Tie range', 
             hue='Utilization', 
             multiple='dodge', 
             palette=[COLOR_DETERMINISTIC, COLOR_CONSTANT], 
             edgecolor=".3",
             linewidth=0.5,
             binwidth=1)
axes.set_yscale('log')
axes.set_title('Edge Utilization - Constant Exposure Subthreshold', fontweight='bold', fontsize=13)
axes.legend(labels=['Constant Exposure Subthreshold', 'Deterministic Activation'], frameon=False)
axes.set_xlim(0, 40)

# Common x-label
axes.set_xlabel('Tie range')
plt.tight_layout()
plt.grid(False)

# Save the figure
plt.savefig('histogram_non_ICM.png', dpi=300)
plt.show()


In [ ]:
# Set up the subplots
fig, axes = plt.subplots(1, 1, figsize=(7, 4), sharex=True)

# Plot for ICM
sns.histplot(data=df[df['Type'] == 'First'], 
             x='Tie range', 
             hue='Utilization', 
             multiple='dodge', 
             palette=[COLOR_DETERMINISTIC, COLOR_FIRST], 
             edgecolor=".3",
             linewidth=0.5,
             binwidth=1)
axes.set_yscale('log')
axes.set_title('Edge Utilization - First Exposure Subthreshold', fontweight='bold', fontsize=13)
axes.legend(labels=['First Exposure Subthreshold', 'Deterministic Activation'], frameon=False)

# Common x-label
axes.set_xlabel('Tie range')
axes.set_xlim(0, 40)
plt.tight_layout()
plt.grid(False)

# Save the figure
plt.savefig('histogram_ICM.png', dpi=300)
plt.show()


In [ ]:
df_spread_time_deterministic_kde = df_spread_time_deterministic[df_spread_time_deterministic['eta'] < 23].copy()
df_spread_time_constant_kde = df_spread_time_constant[df_spread_time_constant['eta'] < 23].copy()
df_spread_time_first_kde = df_spread_time_first[df_spread_time_first['eta'] < 23].copy()

# Plot
plt.figure(figsize=(7, 4))

# KDE curves (with labels)
sns.kdeplot(df_spread_time_deterministic_kde['Spread_time_90'],
            label='Deterministic',
            color=COLOR_DETERMINISTIC,
            linewidth=2,
            clip=(0, 500))

sns.kdeplot(df_spread_time_constant_kde['Spread_time_90'],
            label='Constant Exposure Subthreshold',
            color=COLOR_CONSTANT,
            linewidth=2,
            clip=(0, 500))

sns.kdeplot(df_spread_time_first_kde['Spread_time_90'],
            label='First Exposure Subthreshold',
            color=COLOR_FIRST,
            linewidth=2,
            clip=(0, 500))


# Labels and title
plt.xlabel('Time to Reach 90% Activation', fontsize=11)
plt.ylabel('Density', fontsize=11)
plt.title('Spread Time Distributions Across Models', fontweight='bold', fontsize=13)

# Legend without the dashed lines
plt.legend(loc='upper right', frameon=False, bbox_to_anchor=(1, 1))
plt.grid(False)


plt.tight_layout()
plt.savefig('spread_time_distribution.png', dpi=300, bbox_inches='tight')
plt.show()
